In [1]:
from normality import normalize
import pandas as pd
from thefuzz import fuzz
from thefuzz import process
from countrynames import to_code
import os
import shutil
import datetime


In [2]:



csv_to_fix = "ProductionTradeSupplyFuelOil_EF_20240603.csv"

now = datetime.datetime.now()
formatted_date = now.strftime("%Y%m%d")
folder_path = "%s_Country_Renamed_%s" %(csv_to_fix.split("_")[0], formatted_date)
os.makedirs(folder_path)
geo_df = pd.read_csv("./ISOToEcoInventToolSub/Geographies.csv")
df = pd.read_csv(csv_to_fix)



In [3]:
df['Region/Regional Scope'].unique()
# df['Country of Applicability'] = "GLO"
# df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)

array(['Madagascar', 'Malawi', 'Malaysia', 'Mali', 'Martinique',
       'Mauritania', 'Mauritius', 'Pakistan', 'Palau', 'Panama',
       'Papua New Guinea', 'Paraguay', 'Morocco', 'Mozambique', 'Myanmar',
       'Peru', 'Philippines', 'New Caledonia', 'New Zealand', 'Nicaragua',
       'Niger', 'Nigeria', 'Senegal', 'Serbia', 'Serbia and Montenegro',
       'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia',
       'Sint Maarten (Dutch part)', 'Romania', 'Russian Federation',
       'Rwanda', 'Saudi Arabia', 'Indonesia', 'India', 'Iceland',
       'Ireland', 'Isle of Man', 'Israel', 'Italy',
       "Korea, Dem.Ppl's.Rep.", 'Korea, Republic of', 'Kosovo', 'Iraq',
       'Argentina', 'Armenia', 'Aruba', 'Antigua and Barbuda',
       'Afghanistan', 'Albania', 'Belgium', 'Belize', 'Benin', 'Angola',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Belarus', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau',
       'Guyana', 'Guadeloupe', 'Haiti', 

In [4]:
def normalize_name(country):
    """Clean up a country name before comparison."""
    return normalize(country, latinize=True)

df_ambig = pd.DataFrame(columns=["Input string", "Mapped name", "Mapped code", "Quarantine?"])

def mapping(country:str)->str:
    global df_ambig
    if(result:=to_code(country)) is not None:
        return result
    matching_rows = df_ambig[df_ambig["Input string"].str.contains(country, case=False, na=False)]
    if not matching_rows.empty:
        return country
    else:
        longname = process.extractOne(country, choices=geo_df['Name'], processor=(lambda x: str(x)), scorer=fuzz.token_set_ratio, score_cutoff=0)[0]
        shortname = geo_df[geo_df['Name'] == longname]['Shortname'].iloc[0]
        new_row = pd.DataFrame([{ 
            "Input string": country,
            "Mapped name": longname,
            "Mapped code": shortname}])
        df_ambig = pd.concat([df_ambig, new_row], ignore_index=True)
        return country
    
Region_column_name = 'Region/Regional Scope'
df[Region_column_name] = df[Region_column_name].apply(normalize_name)
df[Region_column_name] =df[Region_column_name].apply(lambda x: mapping(x))

df_ambig.to_csv("ambig.csv", index=False)
df_ambig.to_csv("./%s/AMBIG_%s.csv" %(folder_path, folder_path), index=False)


/var/folders/7z/6g3hfctx56g77d9vf2r0bbx00000gn/T/ipykernel_66539/140740927.py:3: ICUWarning: Install 'pyicu' for better text transliteration.
  return normalize(country, latinize=True)


In [35]:
input("Go to ambig.csv and correct ambiguous inputs")

''

In [5]:
def split_ambig():
    ambig = pd.read_csv('./ambig.csv')
    if ambig.empty:
        return ambig, ambig
    corrected = ambig[ambig['Quarantine?'] != 'x']
    further_quarantined = ambig[ambig['Quarantine?'] == 'x']
    return corrected, further_quarantined

def remove_quarantined(df_first_clean, further_quarantined, corrected):
    overlap_values = further_quarantined['Input string'].unique()
    removed_rows_df = df_first_clean[df_first_clean['Region/Regional Scope'].isin(overlap_values)].copy()
    df_second_clean = df_first_clean[~df_first_clean['Region/Regional Scope'].isin(overlap_values)]
    
    merged_df = pd.merge(df_second_clean, corrected, left_on='Region/Regional Scope', right_on='Input string', how='left')
    merged_df.loc[merged_df['Input string'].notna(), 'Region/Regional Scope'] = merged_df['Mapped code']
    merged_df.drop(columns=['Mapped code', "Input string","Mapped name","Quarantine?"], inplace=True)
    
    file_path = "./ambig.csv"
    if os.path.exists(file_path):
        shutil.move(file_path, "./%s/ambig_VERIFIED_%s.csv" % (folder_path, folder_path))
    return merged_df, removed_rows_df



corrected, further_quarantined = split_ambig()
merged_df, removed_rows_df =remove_quarantined(df, further_quarantined, corrected)




In [6]:
# In this cell, please add any code you'd like to edit 
print(merged_df['Region/Regional Scope'].unique())
print(removed_rows_df['Region/Regional Scope'].unique())


['MG' 'MW' 'MY' 'ML' 'MQ' 'MR' 'MU' 'PK' 'PW' 'PA' 'PG' 'PY' 'MA' 'MZ'
 'MM' 'PE' 'PH' 'NC' 'NZ' 'NI' 'NE' 'NG' 'SN' 'RS' 'CSXX' 'SC' 'SL' 'SG'
 'SK' 'SI' 'SX' 'RO' 'RU' 'RW' 'SA' 'ID' 'IN' 'IS' 'IE' 'IM' 'IL' 'IT'
 'KP' 'KR' 'XK' 'IQ' 'AR' 'AM' 'AW' 'AG' 'AF' 'AL' 'BE' 'BZ' 'BJ' 'AO'
 'AT' 'AZ' 'BS' 'BH' 'BD' 'BY' 'GT' 'GG' 'GN' 'GW' 'GY' 'GP' 'HT' 'GB'
 'TZ' 'US' 'YE' 'ZM' 'VN' 'VE' 'RAS' 'MK' 'NO' 'OM' 'PL' 'PT' 'NA' 'NP'
 ' CS' 'NL' 'ME' 'CL' 'CN' 'HK' 'MO' 'CO' 'CG' 'CK' 'CR' 'AU' 'KH' 'CM'
 'CA' 'BR' 'BN' 'BG' 'BF' 'BI' 'CF' 'TD' 'UG' 'UA' 'FI' 'AE' 'FR' 'TR'
 'TM' 'EC' 'EG' 'SV' 'ER' 'EE' 'PR' 'QA' 'MD' 'GR' 'GL' 'GD' 'JP' 'JE'
 'JO' 'KZ' 'KE' 'UY' 'UZ' 'VU' ' CSRR' 'LA' 'LV' 'MT' 'LB' 'LR' 'LY' 'LT'
 'MX' 'LU' 'MN' 'KW' 'KG' 'RE' 'TT' 'TN' 'NR' 'BB' 'FO' 'FJ' 'ET' 'SZ'
 'SR' 'SE' 'CH' 'SY' 'TJ' 'TH' 'TG' 'TO' 'GE' ' DD' 'DE' 'GH' 'GI' 'GA'
 'GM' 'IR' 'DM' 'DO' 'DK' 'GF' 'PF' 'JM' 'ES' 'LK' 'LC' 'PS' 'SD' 'CZ'
 'CD' 'DJ' 'HN' 'HU' 'ZA' 'SO' 'DZ' 'AS' 'CU' 'CW' 'CY' 'BQ' 'BA' 'BM

In [27]:
input("Look through each dataset and adjust any inconsistent values")




''

In [53]:
removed_rows_df.to_csv("./%s/%s_further_quarantined.csv" %(folder_path, folder_path), index=False) 
merged_df.to_csv("./%s/%s.csv"  %(folder_path, folder_path), index=False)


In [54]:
shutil.copy("./Nationalizer.ipynb", "%s" % folder_path)
shutil.move(csv_to_fix, "%s" % folder_path)
shutil.make_archive(folder_path.split('.')[0], 'zip', folder_path)
shutil.rmtree(folder_path)
